# Setting boundary conditions in PorePy
In this tutorial we will visit how to set boundary conditions for different model problems in PorePy.
In general, the setup is roughly as follows:
* Boundary condition type (Neumann or Dirichlet) is set by one method
* Boundary condition values are set through one or more methods

Internally in the code there are Neumann and Dirichlet filters being assigned according to the user's input in the boundary condition type method.
These filters make sure that only values set by a Dirichlet-method is assigned to Dirichlet boundaries, and similar for the Neumann case. For explaining this more thoroughly we will present some examples.


# The flow problem

# The thermal problem

# The mechanics problem
Similar to the previous problems, we need to define one method for the type of boundary condition we are interested in, and another one for assigning the values. 
In the case of the mechanics problem we utilize the momentum balance model class. 
The momentum balance model class has the following methods predefined for setting boundary condition type and values:
* `bc_type_mechanics` for setting boundary condition type.
* `bc_values_displacement` for Dirichlet values.
* `bc_values_stress` for Neumann values.

As opposed to with the previous examples, this is a vector problem. 
Therefore we will here have to assign vectorial boundary condition values. 
This is shown below:


In [22]:
import numpy as np
import porepy as pp

class ModifiedBoundaryConditions:
    def bc_type_mechanics(self, sd: pp.Grid) -> pp.BoundaryConditionVectorial:
        bounds = self.domain_boundary_sides(sd)

        bc = pp.BoundaryConditionVectorial(sd, bounds.west, "dir")
        return bc

    def bc_values_stress(self, bg: pp.BoundaryGrid) -> np.ndarray:
        """Setting stress boundary conditions at north, south and east boundaries.
        
        Specifically, we assign different values for the x- and y-component of the boundary value vector.

        """
        values = np.ones((self.nd, bg.num_cells))
        bounds = self.domain_boundary_sides(bg)

        values[0][bounds.north + bounds.south] *= 1.5
        values[1][bounds.north + bounds.south] *= 0.5

        return values.ravel("F")

    def bc_values_displacement(self, bg: pp.BoundaryGrid) -> np.ndarray:
        """Setting displacement boundary conditions.
        
        This method returns an array of boundary condition values with the value 5 for all boundary x-components. However, since only the western boundary has the Dirichlet filter, the internal code will only consider the value 5 for the western boundary.
        
        """
        values = np.ones((self.nd, bg.num_cells))
    
        values[0] *= 5
        return values.ravel("F")
